# Banking System Implementation

#### Tier 1: Core Transaction Management
- Create an in-memory banking database system
- Implement basic CRUD operations for accounts
- Record and handle deposits and withdrawals
- Implement transfer functionality between accounts
- Ensure atomic operations (transfers must be accepted before processing)
- Support transaction logging

#### Tier 2: Data Analytics & Metrics
- Implement function that uses an efficient ranking algorithm to return top K accounts by: 
  - Total transaction volume (deposits + withdrawals)
  - Total outgoing money
- Track both incoming and outgoing transactions
- Support data metrics and analysis

#### Tier 3: Advanced Transaction Features
- Implement scheduled transactions system - can schedule a transaction at given time, and cancel it later 

#### Tier 4: Account Management
- Implement account merging functionality - merge two accounts while maintaining separate account histories
- Preserve separate transaction histories when merging, maintain data integrity during merges

#### Follow-up Considerations
- How would you scale this system?
- How would you handle concurrent transactions?
- How would you implement disaster recovery?
- How would you optimize the ranking algorithm for large datasets?
- How would you ensure data consistency with scheduled transactions?

In [81]:
# STEP 1: Basic banking system
from pydantic import BaseModel, Field, validator, NonNegativeFloat
import threading

class InsufficientFundsError(Exception):
    def __init__(self, new_balance: float, account_id: int):
        super().__init__(f"Insufficient funds error: new balance {new_balance} would be <0 for account_id {account_id}")

class AccountNotFoundError(Exception):
    def __init__(self, account_id: int):
        super().__init__(f"Get account failed: no account exists for ID {account_id}")

class BankAccount(BaseModel):
    account_id: int = Field(..., description="Account ID, unique integer", ge=0)
    owner: str = Field(..., description="Account owner's name", min_length=1)  # Must be non-empty str
    balance: NonNegativeFloat = Field(..., description="Account balance in $USD")

    @validator('owner')
    def owner_validator(cls, v):
        if not v or v == '':
            raise ValueError('owner cannot be empty')
        return v

class Bank:
    def __init__(self):
        # in memory-database: banks store accounts, which map IDs to account details
        self.accounts: dict[int, BankAccount] = {}
        self.curr_id: int = 0  # uuids are just ints, which increment up over time
        # locks
        self.account_locks: dict[int, threading.Lock] = {}
        self.global_lock = threading.Lock()  # lock for all accounts modifications

    def create_account(self, owner: str, starting_balance: float) -> BankAccount:
        """Create bank account with owner and balance, using IDs as account identifiers"""
        with self.global_lock:  #  only allow modifying 1 account in dict at a time
            account_id = self.curr_id
            self.curr_id += 1
            self.account_locks[account_id] = threading.Lock()  # lock for this account
            account = BankAccount(owner=owner, balance=starting_balance, account_id=account_id)
            self.accounts[account_id] = account
        print(f"Account {account_id} created successfully for {owner} with balance ${starting_balance}")
        return self.accounts[account_id]

    def read_account(self, account_id: int) -> BankAccount:
        """Return the BankAccount for account with ID - read-only"""
        if not self.accounts[account_id]:
            raise AccountNotFoundError(account_id)
        return self.accounts[account_id]

    def read_all_accounts(self) -> dict[int, BankAccount]:
        """Return all of the bank accounts in the bank - read-only"""
        return self.accounts

    def deposit(self, account_id: int, amount: float) -> float:
        """Add a positive amount to the specified account ID"""
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        if account_id not in self.accounts:
            raise AccountNotFoundError(account_id)
        with self.account_locks[account_id]:  # 1 transaction for account at a time
            new_balance = self.accounts[account_id].balance + amount
            self.accounts[account_id].balance = new_balance
            print(f"Deposited ${amount} to account {account_id}. New balance: ${new_balance}")
            return new_balance

    def withdraw(self, account_id: int, amount: float) -> float:
        """Subtract a positive amount from the specified account ID"""
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive")
        if account_id not in self.accounts:
            raise AccountNotFoundError(account_id)
        with self.account_locks[account_id]:  # 1 transaction for account at a time
            new_balance = self.accounts[account_id].balance - amount
            if new_balance < 0:
                raise InsufficientFundsError(new_balance, account_id)
            self.accounts[account_id].balance = new_balance
            print(f"Withdrew ${amount} from account {account_id}. New balance: ${new_balance}")
            return new_balance

    def delete_account(self, account_id: int) -> bool:
        """Delete the account with ID, returning the deleted bank account"""
        if account_id not in self.accounts:
            raise AccountNotFoundError(account_id)
        with self.global_lock:  # only allow 1 account deletion at a time
            del self.accounts[account_id]
            del self.account_locks[account_id]
            return True

    def transfer(self, from_account_id: int, to_account_id: int, amount: float) -> tuple[float, float]:
        """Transfer money between accounts, returning the new balances of both accounts."""
        if from_account_id == to_account_id:
            raise ValueError(f"Transfer failed: cannot transfer from account {from_account_id} to itself")
        if amount <= 0:
            raise ValueError(f"Transfer failed: transfer amount {amount} must be positive")
        with self.global_lock:  # only allow 1 transfer at a time
            # verify accounts exist before proceeding
            if from_account_id not in self.accounts:
                raise AccountNotFoundError(from_account_id)
            if to_account_id not in self.accounts:
                raise AccountNotFoundError(to_account_id)
            if self.accounts[from_account_id].balance < amount:
                raise InsufficientFundsError(self.accounts[from_account_id].balance - amount, from_account_id)

        # perform the transfer atomically
        try:
            from_account = self.accounts[from_account_id]
            to_account = self.accounts[to_account_id]

            from_account.balance -= amount
            to_account.balance += amount

            print(f"""Successfully transferred ${amount} from account {from_account_id}
                  (new balance: ${from_account.balance}) to {to_account_id}
                  (new balance: ${to_account.balance})""")

            return (from_account.balance, to_account.balance)

        except Exception as e:
                # if anything fails, roll back the changes
                if 'from_account' in locals():
                    from_account.balance += amount  # restore original balance
                raise ValueError(f"Transfer failed: {str(e)}")

/var/folders/m3/mn85ncps6dg556xrp9nl8gwc0000gn/T/ipykernel_27943/3482062590.py:18: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @validator('owner')


In [85]:
bank_of_america = Bank()

j = bank_of_america.create_account('Jeremy', 21000)
l = bank_of_america.create_account('Lisa', 10000)
k = bank_of_america.create_account('Bill', 550)
m = bank_of_america.create_account('Tom', 30)

Account 0 created successfully for Jeremy with balance $21000
Account 1 created successfully for Lisa with balance $10000
Account 2 created successfully for Bill with balance $550
Account 3 created successfully for Tom with balance $30


In [89]:
# test atomicity of transfers
import threading
import time
import random

def test_atomic_transfer(bank, from_id, to_id, amount):
    """Test a single transfer and verify atomicity"""
    # verify accounts exist before accessing balances
    if from_id not in bank.accounts or to_id not in bank.accounts:
        print(f"Transfer failed: Account not found")
        return False

    initial_from = bank.accounts[from_id].balance
    initial_to = bank.accounts[to_id].balance

    try:
        new_balances = bank.transfer(from_id, to_id, amount)

        # verify returned balances match actual account balances
        assert new_balances[0] == bank.accounts[from_id].balance
        assert new_balances[1] == bank.accounts[to_id].balance

        # verify balances changed by exactly the transfer amount
        assert bank.accounts[from_id].balance == initial_from - amount
        assert bank.accounts[to_id].balance == initial_to + amount

        # verify total money in system remained constant
        assert initial_from + initial_to == bank.accounts[from_id].balance + bank.accounts[to_id].balance

        return True

    except Exception as e:
        # on failure, verify no partial changes occurred
        assert bank.accounts[from_id].balance == initial_from
        assert bank.accounts[to_id].balance == initial_to
        print(f"Transfer failed as expected: {e}")
        return False

# Test 1: Basic successful transfer
print("\nTest 1: Basic successful transfer")
assert test_atomic_transfer(bank_of_america, j.account_id, l.account_id, 1000)

# Test 2: Transfer with insufficient funds
print("\nTest 2: Transfer with insufficient funds")
assert not test_atomic_transfer(bank_of_america, k.account_id, m.account_id, 1000000)

# Test 3: Transfer to non-existent account
print("\nTest 3: Transfer to non-existent account")
assert not test_atomic_transfer(bank_of_america, j.account_id, 99999, 100)

# Test 4: Transfer from non-existent account
print("\nTest 4: Transfer from non-existent account")
assert not test_atomic_transfer(bank_of_america, 99999, l.account_id, 100)

# Test 5: Transfer of negative amount
print("\nTest 5: Transfer of negative amount")
assert not test_atomic_transfer(bank_of_america, j.account_id, l.account_id, -100)

# Test 6: Transfer to same account
print("\nTest 6: Transfer to same account")
assert not test_atomic_transfer(bank_of_america, j.account_id, j.account_id, 100)

# Test 7: Multiple rapid transfers between same accounts
print("\nTest 7: Multiple rapid transfers between same accounts")
initial_total = (bank_of_america.accounts[j.account_id].balance +
                bank_of_america.accounts[l.account_id].balance)

for _ in range(50):
    amount = random.randint(1, 1000)
    if random.random() < 0.5:
        test_atomic_transfer(bank_of_america, j.account_id, l.account_id, amount)
    else:
        test_atomic_transfer(bank_of_america, l.account_id, j.account_id, amount)

final_total = (bank_of_america.accounts[j.account_id].balance +
               bank_of_america.accounts[l.account_id].balance)
assert initial_total == final_total

# Test 8: Zero amount transfer
print("\nTest 8: Zero amount transfer")
assert not test_atomic_transfer(bank_of_america, j.account_id, l.account_id, 0)

print("\nAll atomicity tests passed successfully!")


Test 1: Basic successful transfer
Updated account 0 balance to 9020.0
Updated account 1 balance to 18000.0
Successfully transferred $1000 from account 0 (new balance: $9020.0) to 1 (new balance: $18000.0)

Test 2: Transfer with insufficient funds
Transfer failed as expected: Insufficient funds error: new balance -995650.0 would be <0 for account_id 2

Test 3: Transfer to non-existent account
Transfer failed: Account not found

Test 4: Transfer from non-existent account
Transfer failed: Account not found

Test 5: Transfer of negative amount
Updated account 0 balance to 9120.0
Updated account 1 balance to 17900.0
Successfully transferred $-100 from account 0 (new balance: $9120.0) to 1 (new balance: $17900.0)


AssertionError: 